In [1]:
# Append rolling_ta to sys path.
import sys
import os

# Assuming rolling_ta is in the same level as tests/
module_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add the path to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from ta.trend import SMAIndicator
from rolling_ta.trend import NumbaSMA
from rolling_ta.data import CSVLoader

In [3]:
csv_loader = CSVLoader()
data = csv_loader.read_resource()

DEBUG - CSVLoader: Loading from resources/btc_ohlcv.csv


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  10000 non-null  float64
 1   open       10000 non-null  float64
 2   high       10000 non-null  float64
 3   low        10000 non-null  float64
 4   close      10000 non-null  float64
 5   volume     10000 non-null  float64
dtypes: float64(6)
memory usage: 468.9 KB


In [5]:
NumbaSMA(data)

DEBUG - bytecode dump:
>          0	NOP(arg=None, lineno=73)
           2	RESUME(arg=0, lineno=73)
           4	LOAD_GLOBAL(arg=1, lineno=78)
          14	LOAD_FAST(arg=0, lineno=78)
          16	LOAD_ATTR(arg=2, lineno=78)
          36	LOAD_FAST(arg=1, lineno=78)
          38	CALL(arg=2, lineno=78)
          46	STORE_FAST(arg=2, lineno=78)
          48	LOAD_CONST(arg=1, lineno=79)
          50	STORE_FAST(arg=3, lineno=79)
          52	LOAD_GLOBAL(arg=5, lineno=80)
          62	LOAD_ATTR(arg=6, lineno=80)
          82	LOAD_FAST(arg=1, lineno=80)
          84	CALL(arg=1, lineno=80)
          92	GET_ITER(arg=None, lineno=80)
>         94	FOR_ITER(arg=10, lineno=80)
          98	STORE_FAST(arg=4, lineno=80)
         100	LOAD_FAST(arg=3, lineno=81)
         102	LOAD_FAST(arg=0, lineno=81)
         104	LOAD_FAST(arg=4, lineno=81)
         106	BINARY_SUBSCR(arg=None, lineno=81)
         110	BINARY_OP(arg=13, lineno=81)
         114	STORE_FAST(arg=3, lineno=81)
         116	JUMP_BACKWARD(arg=

In [14]:
%%timeit
for _ in range(10000):
    sma = SMAIndicator(data["close"], 14)
    sma.sma_indicator()

2.56 s ± 61.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
for _ in range(10000):
    NumbaSMA(data)

436 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
slice_a = data.copy().iloc[:100]
slice_b = data.copy().iloc[100:]

In [16]:
%%timeit
for _ in range(5):
    rolling = SMA(slice_a)
    for _, series in slice_b.iterrows():
        rolling.update(series)

8.25 s ± 22.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
for _ in range(5):
    rolling = NumbaSMA(slice_a)
    for _, series in slice_b.iterrows():
        rolling.update(series)

1.51 s ± 26.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
